## This Notebook reads in the meeting summaries and does all required preprocessing prior to model invocation

The end result is training and test data split. Each dataset will be an array of three numpy arrays (0) sentence encodings (1) masks to identify paddings (2) numbered sequence ids (line numbers). A separate 

In [1]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import torch
from time import time
import io
import re
from csv import reader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from datetime import datetime
import glob, os

In [2]:
DATA_DIR = '/home/rajivn/W266/W266-fall-2020-hwu-rnair/data/ICSI_plus_NXT/processing'

In [3]:
from transformers import BertTokenizer, TFBertModel

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
tokenizer.tokenize('I\'ll learn to swim in 12342 years.')
tokenizer.convert_tokens_to_ids([
    '[CLS]', 'i', "'",'ll', 'learn','to','swim','in','123', '##42', 'years', '.'])


[101, 178, 112, 1325, 3858, 1106, 11231, 1107, 13414, 23117, 1201, 119]

In [5]:
max_len = 0
sentences = []
for inp_file in sorted(glob.glob(DATA_DIR+"/*txt")):
    with open(inp_file, "r") as f:
        lines = [line.split("|") for line in f.readlines()]
        for line, y in lines:
            input_ids = tokenizer.encode(line, add_special_tokens=True)
            max_len=max(max_len, len(input_ids))
            sentences.extend(line)


In [6]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation=True,
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

/home/rajivn/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1767: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


KeyboardInterrupt: 